# Tutorial 1: A First Example - Function Inversion on a Circle

The aim of this tutorial is to show how this library can be used to solve a simple, but non-trivial, linear inverse problem from start to finish. We will not explain every detail, as later tutorials will build up the various ideas.

### The Problem

We consider a smooth function, $u$, defined on the unit circle, $\mathbb{S}^{1}$. We are given $n$ noisy point measurements of this function:

$$
d_{i} = u(\theta_{i}) + e_{i}
$$

where the $\theta_{i}$ are known locations and the $e_{i}$ are normally distributed random errors. From this data, we wish to estimate the unknown function, $u$.

We will solve this problem using two different methods: a **Minimum Norm Inversion** and a full **Bayesian Inversion**.

## 1. Defining the Problem Components

First, we define the three key components of any inverse problem:
1.  **The Model Space:** The `HilbertSpace` our unknown function `u` lives in. We'll use a `Sobolev` space, which is a space of smooth functions.
2.  **The Forward Operator:** The `LinearOperator` that maps a model `u` to the "perfect" data `d_perfect = A(u)`. Here, this is the `point_evaluation_operator`.
3.  **The Data Error:** A `GaussianMeasure` that describes the statistics of the noise `e`.

These are then bundled into a `LinearForwardProblem` object.

In [ ]:
# To run in colab, uncomment the line below to install pygeoinf. 
#%pip install pygeoinf

import numpy as np
import matplotlib.pyplot as plt
import pygeoinf as inf
from pygeoinf.symmetric_space.circle import Sobolev, CircleHelper

# For reproducibility
np.random.seed(42)

# 1. Set up the model space
# We use a helper method to automatically determine the necessary resolution (kmax)
# from the desired physical properties: a smoothness order of 2.0 and a
# correlation length-scale of 0.05.
model_space = Sobolev.from_sobolev_parameters(2.0, 0.05)

# 2. Define the forward operator
# This operator will take a function from our model_space and evaluate it
# at `n` random points on the circle.
n_data = 20
observation_points = model_space.random_points(n_data)
forward_operator = model_space.point_evaluation_operator(observation_points)
data_space = forward_operator.codomain

# 3. Define the data error measure
# We assume the noise has a standard deviation of 0.1.
standard_deviation = 0.1
data_error_measure = inf.GaussianMeasure.from_standard_deviation(
    data_space, standard_deviation
)

# Bundle everything into a forward problem object
forward_problem = inf.LinearForwardProblem(
    forward_operator, data_error_measure=data_error_measure
)

print(f"Model space dimension (kmax): {model_space.kmax}")
print(f"Data space dimension: {data_space.dim}")

## 2. Generating Synthetic Data

To test our inversion methods, we need a "true" model and a corresponding set of noisy data. We create the true model by drawing a random sample from a "prior" `GaussianMeasure`. This prior represents our initial guess about what the function might look like (in this case, a smooth function with a typical amplitude of 1.0).

In [ ]:
# Define a prior measure on the model space.
# We use a heat kernel covariance, which produces smooth, random functions.
model_prior_measure = model_space.point_value_scaled_heat_kernel_gaussian_measure(0.1, 1.0)

# Generate the true model and the corresponding noisy data.
true_model, data = forward_problem.synthetic_model_and_data(model_prior_measure)

### A Helper Function for Plotting

To avoid repeating code, let's create a helper function to visualize our results. It will plot the true model, the noisy data, the inversion result, and an optional uncertainty bound.

In [ ]:
def plot_results(
    space: CircleHelper,
    true_model: np.ndarray,
    data: np.ndarray,
    obs_points: np.ndarray,
    data_std: float,
    solution_model: np.ndarray,
    solution_label: str,
    solution_std: np.ndarray = None,
):
    """Helper function to create a consistent plot."""
    fig, ax = space.plot(true_model, color="k", linestyle="--", label="True Model", figsize=(15, 10))
    
    # Plot the solution
    space.plot(solution_model, fig=fig, ax=ax, color="b", label=solution_label)
    
    # Plot uncertainty bounds if provided
    if solution_std is not None:
        space.plot_error_bounds(
            solution_model, 2 * solution_std, fig=fig, ax=ax, alpha=0.2, color="b"
        )
        
    # Plot the noisy data points
    ax.errorbar(obs_points, data, 2 * data_std, fmt="ko", capsize=3, label="Data")
    
    ax.set_title("Inversion Results", fontsize=16)
    ax.set_xlabel("Angle (radians)")
    ax.set_ylabel("Function Value")
    ax.legend()
    ax.grid(True, linestyle=":", alpha=0.7)
    
    plt.show()

# Plot our initial ground truth and data
plot_results(model_space, true_model, data, observation_points, standard_deviation,
             solution_model=np.zeros_like(true_model), solution_label="Initial State (Zero)")

## 3. Method 1: Minimum Norm Solution

Our first approach is the **minimum norm solution**. Of all the models that fit the data acceptably (according to a chi-squared test), this method finds the unique one with the smallest Sobolev norm. It yields a single, deterministic estimate of the model.

We use `CGSolver` because the normal equations for this problem are posed on the high-dimensional model space. A matrix-free iterative solver is much more efficient than forming a large, dense matrix.

In [ ]:
# Set up the inversion method
minimum_norm_inversion = inf.LinearMinimumNormInversion(forward_problem)

# Get the operator that maps data to the solution.
solver = inf.CGSolver()
minimum_norm_operator = minimum_norm_inversion.minimum_norm_operator(solver)

# Compute the minimum norm model
minimum_norm_model = minimum_norm_operator(data)

# Plot the result
plot_results(
    model_space,
    true_model,
    data,
    observation_points,
    standard_deviation,
    solution_model=minimum_norm_model,
    solution_label="Minimum Norm Solution",
)

## 4. Method 2: Bayesian Inversion

For the second method, we perform a **Bayesian inversion**. This requires a prior measure, and we'll use the same one we used to generate our true model. Instead of a single solution, this method returns a full posterior probability distribution (`GaussianMeasure`), which includes both a best-estimate (the posterior mean) and a quantification of uncertainty (the posterior covariance).

Since the full posterior covariance operator is too large to store, we create a low-rank approximation of it. We can then draw samples from this approximation to estimate the pointwise uncertainty.

The key matrix to invert in the Bayesian case lives on the low-dimensional data space. Since we only have 20 data points, a direct `CholeskySolver` is very fast and efficient.

In [ ]:
# Set up the Bayesian inversion method
bayesian_inversion = inf.LinearBayesianInversion(forward_problem, model_prior_measure)

# Compute the posterior distribution
solver = inf.CholeskySolver(galerkin=True)
model_posterior_measure = bayesian_inversion.model_posterior_measure(data, solver)
posterior_mean = model_posterior_measure.expectation

# Estimate the pointwise variance from a low-rank approximation
low_rank_posterior = model_posterior_measure.low_rank_approximation(
    10, method="variable", rtol=1e-4
)
posterior_pointwise_variance = low_rank_posterior.sample_pointwise_variance(200)
posterior_std = np.sqrt(posterior_pointwise_variance)

# Plot the posterior mean and its 95% confidence interval
plot_results(
    model_space,
    true_model,
    data,
    observation_points,
    standard_deviation,
    solution_model=posterior_mean,
    solution_label="Posterior Mean",
    solution_std=posterior_std,
)

## 5. Conclusion

As you can see, both methods do a good job of recovering the true model from the sparse, noisy data.

* The **Minimum Norm Solution** provides a single, good-looking estimate.
* The **Bayesian Inversion** provides a similar estimate (the posterior mean) but also gives us the credible interval (the blue shaded region), which tells us where our estimate is most and least certain. This uncertainty quantification is a key advantage of the Bayesian approach, but one that depends on the prior information being suitable. 